# Install Libraries

In [1]:
!pip install langchain -q
!pip install sentence-transformers -q
!pip install faiss-gpu -q
!pip install pypdf2 -q
!pip install langchain_community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.3 MB/s eta 0:00:00


In [2]:
!pip install langchain-groq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


# Load LLM

In [3]:
from dotenv import load_env
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

In [4]:
load_env()

In [5]:
llm = ChatGroq(model_name="llama3-8b-8192", temperature=0.7)

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."),
     ("human", "{text}")]
)

In [7]:
chain = prompt | llm

In [8]:
chain.invoke({"text": "What is the capital of india?"})

AIMessage(content='The capital of India is New Delhi.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 28, 'total_tokens': 36, 'completion_time': 0.006289845, 'prompt_time': 0.011356395, 'queue_time': None, 'total_time': 0.01764624}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5ce2ab1-9209-433e-98bd-b96a51a6b473-0')

# Load PDF

In [9]:
from PyPDF2 import PdfReader

In [10]:
def read_pdf(pdf):
  raw_text = ""
  file = PdfReader(pdf)
  for i, page in enumerate(file.pages):
    content = page.extract_text()
    if content:
      raw_text += content
  return raw_text

In [11]:
raw_text = read_pdf("/content/drive/MyDrive/Colab Notebooks/Learning LangChain/frbm1.pdf")

In [12]:
raw_text[:500]

'Statements of Fiscal Policy as required under \nthe Fiscal Responsibility and Budget \nManagement Act, 2003\nMINISTRY OF FINANCE\nBUDGET DIVISIONNirmala Sitharaman\nMinister of Finance\nFebruary , 2024ºÉiªÉàÉä´É VÉªÉiÉä\nGOVERNMENT OF INDIATABLE OF CONTENTS\nSl. No.  Statements Page No.\nPreface\n1 Macro-Economic Framework Statement\n2 Medium Term Fiscal Policy cum Fiscal Policy Strategy\nStatement(i)\n1\n5PREF ACE\nThe Fiscal Responsibility and Budget Management Act, 2003 was enacted with\na view to provide a '

In [13]:
len(raw_text)

52740

# Split the text using RecursiveCharcterTextSplitter

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    separators=["\n"]
)

In [17]:
texts = text_splitter.create_documents([raw_text])

In [18]:
texts[:5]

[Document(page_content='Statements of Fiscal Policy as required under \nthe Fiscal Responsibility and Budget \nManagement Act, 2003\nMINISTRY OF FINANCE\nBUDGET DIVISIONNirmala Sitharaman\nMinister of Finance'),
 Document(page_content='Minister of Finance\nFebruary , 2024ºÉiªÉàÉä´É VÉªÉiÉä\nGOVERNMENT OF INDIATABLE OF CONTENTS\nSl. No.  Statements Page No.\nPreface\n1 Macro-Economic Framework Statement'),
 Document(page_content='2 Medium Term Fiscal Policy cum Fiscal Policy Strategy\nStatement(i)\n1\n5PREF ACE\nThe Fiscal Responsibility and Budget Management Act, 2003 was enacted with'),
 Document(page_content='a view to provide a legislative framework for reduction of deficit and thereby debt, of\nthe Central Government to a sustainable level over a medium term so as to ensure'),
 Document(page_content='inter-generational equity in fiscal management and long term macro-economic stability.\nThe Fiscal Responsibility and Budget Management Act, 2003 and the Fiscal')]

In [19]:
len(texts)

312

# Import Embedding Model

In [20]:
from langchain.embeddings import HuggingFaceEmbeddings

In [21]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
 model_name=modelPath,
 model_kwargs=model_kwargs,
 encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
text = "Hello World"
result = embeddings.embed_query(text)
result[:10]

[-0.034477271139621735,
 0.03102320246398449,
 0.006734997034072876,
 0.026108991354703903,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692403554916382,
 -0.006441450212150812,
 -0.04745049029588699,
 0.014758898876607418]

In [23]:
len(result)

384

# Store embedding in vectorstore

In [24]:
from langchain.vectorstores import FAISS

In [25]:
db = FAISS.from_documents(texts, embeddings)

In [27]:
db.similarity_search("financial budget fy 2024-2025")

[Document(page_content='budgeted `10 lakh crore of Capital Expenditure for\nFY 2023-24, of which more than 58 per cent has been\nspent from April to November 2023. Revenue\nexpenditure has shown an increase of 3.6 per cent'),
 Document(page_content='Capital Expenditure\n26. Budget for FY 2024-25 has an outlay of about\n`11,11,111 crore (3.4 per cent of GDP) for capital\nexpenditure. The budgeted capital expenditure is almost'),
 Document(page_content='Expenditure\n25. In the Budget Estimates of FY 2024-25, total\nexpenditure is pegged at `47.66 lakh crore. The\nexpenditure is expected to grow by 6.1 per cent over\nRE 2023-24.\nCapital Expenditure'),
 Document(page_content='Deficit to 9.2 per cent of GDP in FY 2020-21 as against\n3.5 per cent of GDP estimated for BE 2020-21.\n62. Budget for FY 2024-25 is being presented at a')]

In [28]:
db

#QandA ChatBot

In [29]:
from langchain.chains import RetrievalQA

In [30]:
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [31]:
def ask_rag(question):
  out = chain.invoke(question)
  return out["result"]

In [32]:
ask_rag("what will be the indian budget in 2024")

'Based on the provided context, it is mentioned that the expenditure during April-November 2023 stood at 5.92 lakh crore, which is 58.5 per cent of the Budget Estimates and 31 per cent higher than the corresponding period of FY 2024-25.\n\nHowever, the exact Indian budget for 2024 is not explicitly mentioned in the provided text.'